In [27]:
import findspark
findspark.init()

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import when,lit,count,isnan,col

In [29]:
from pyspark import SparkContext, SparkConf
# Creating a spark context class
spark = SparkSession.builder.appName("Random Forest Regression").getOrCreate()

In [30]:
data = spark.read.csv('data.csv', inferSchema = True, header = True)
data.show(5)

+----+----------+----+--------------------+---------+----------------+-----------------+----------------+---------------+--------------------+------------+-------------+-----------+--------+----------+-----+
|Make|     Model|Year|    Engine Fuel Type|Engine HP|Engine Cylinders|Transmission Type|   Driven_Wheels|Number of Doors|     Market Category|Vehicle Size|Vehicle Style|highway MPG|city mpg|Popularity| MSRP|
+----+----------+----+--------------------+---------+----------------+-----------------+----------------+---------------+--------------------+------------+-------------+-----------+--------+----------+-----+
| BMW|1 Series M|2011|premium unleaded ...|      335|               6|           MANUAL|rear wheel drive|              2|Factory Tuner,Lux...|     Compact|        Coupe|         26|      19|      3916|46135|
| BMW|  1 Series|2011|premium unleaded ...|      300|               6|           MANUAL|rear wheel drive|              2|  Luxury,Performance|     Compact|  Convertible

In [31]:
data.printSchema()

root
 |-- Make: string (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Engine Fuel Type: string (nullable = true)
 |-- Engine HP: integer (nullable = true)
 |-- Engine Cylinders: integer (nullable = true)
 |-- Transmission Type: string (nullable = true)
 |-- Driven_Wheels: string (nullable = true)
 |-- Number of Doors: integer (nullable = true)
 |-- Market Category: string (nullable = true)
 |-- Vehicle Size: string (nullable = true)
 |-- Vehicle Style: string (nullable = true)
 |-- highway MPG: integer (nullable = true)
 |-- city mpg: integer (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- MSRP: integer (nullable = true)



In [32]:
data.describe().toPandas()

summary   Make               Model                Year  Engine Fuel Type  \
0   count  11914               11914               11914             11911   
1    mean   None   745.5822222222222   2010.384337753903              None   
2  stddev   None  1490.8280590623795  7.5797398875957995              None   
3     min  Acura            1 Series                1990            diesel   
4     max  Volvo                  xD                2017  regular unleaded   

            Engine HP   Engine Cylinders Transmission Type     Driven_Wheels  \
0               11845              11884             11914             11914   
1  249.38607007176023  5.628828677213059              None              None   
2  109.19187025917194   1.78055934824622              None              None   
3                  55                  0  AUTOMATED_MANUAL   all wheel drive   
4                1001                 16           UNKNOWN  rear wheel drive   

      Number of Doors     Market Category Vehicle Size  Vehicle Style  \
0               11908               11914        11914          11914   
1  3.4360933825999327                None         None           None   
2  0.8813153865835529                None         None           None   
3                   2           Crossover      Compact  2dr Hatchback   
4                   4  Performance,Hybrid      Midsize          Wagon   

          highway MPG           city mpg          Popularity  \
0               11914              11914               11914   
1  26.637485311398354  19.73325499412456  1554.9111969111968   
2   8.863000766979422  8.987798160299237  1441.8553466274648   
3                  12                  7                   2   
4                 354                137                5657   

                 MSRP  
0               11914  
1  40594.737032063116  
2   60109.10360365422  
3                2000  
4             2065902

In [34]:
def replace(column, value):
    return when(column!=value,column).otherwise(lit(None))

In [35]:
data = data.withColumn("Market Category", replace(col("Market Category"),"N/A"))

In [33]:
data.select('Market Category').show(600,truncate=False)

+-------------------------------------+
|Market Category                      |
+-------------------------------------+
|Factory Tuner,Luxury,High-Performance|
|Luxury,Performance                   |
|Luxury,High-Performance              |
|Luxury,Performance                   |
|Luxury                               |
|Luxury,Performance                   |
|Luxury,Performance                   |
|Luxury,High-Performance              |
|Luxury                               |
|Luxury                               |
|Luxury,High-Performance              |
|Luxury,Performance                   |
|Luxury,Performance                   |
|Luxury                               |
|Luxury,Performance                   |
|Luxury,High-Performance              |
|Luxury,High-Performance              |
|Luxury                               |
|Luxury                               |
|Luxury                               |
|Luxury                               |
|Luxury                               |


In [36]:
data.select([count(when(isnan(c)|col(c).isNull(),c)).alias(c) for c in data.columns]).show()

+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|Make|Model|Year|Engine Fuel Type|Engine HP|Engine Cylinders|Transmission Type|Driven_Wheels|Number of Doors|Market Category|Vehicle Size|Vehicle Style|highway MPG|city mpg|Popularity|MSRP|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+
|   0|    0|   0|               3|       69|              30|                0|            0|              6|           3742|           0|            0|          0|       0|         0|   0|
+----+-----+----+----------------+---------+----------------+-----------------+-------------+---------------+---------------+------------+-------------+-----------+--------+----------+----+



In [38]:
data.count()

11914

In [39]:
#deleting the column Market Category
data = data.drop("Market Category")
# deleting the all null values 
data = data.na.drop()

In [14]:
print((data.count(), len(data.columns)))


(11914, 16)


In [40]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ["Year","Engine HP","Engine Cylinders",
                                        "Number of Doors","highway MPG","city mpg","Popularity"],
                           outputCol = "Input Attributes")

In [41]:
from pyspark.ml.regression import RandomForestRegressor
regressor = RandomForestRegressor(featuresCol = 'Input Attributes', labelCol = "MSRP")

In [18]:
from pyspark.ml import Pipeline
pipeline  = Pipeline(stages = [assembler,regressor])
#--Saving the Pipeline
#pipeline.write().overwrite().save("pipeline_save")

In [ ]:
#System.setProperty("hadoop.home.dir", "/")

In [43]:
from pyspark.ml.tuning import ParamGridBuilder
paramGrid = ParamGridBuilder().addGrid(regressor.numTrees,[100,500]).build()

In [44]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

crossval = CrossValidator(estimator = pipeline,
                         estimatorParamMaps = paramGrid,
                         evaluator = RegressionEvaluator(labelCol = "MSRP"),
                         numFolds = 3)

In [45]:
train_data , test_data = data.randomSplit([0.8,0.2], seed = 123)

In [46]:
cvModel = crossval.fit(train_data)

In [47]:
bestModel = cvModel.bestModel
print(bestModel.stages)

[VectorAssembler_8ef0372a7487, RandomForestRegressionModel: uid=RandomForestRegressor_b336bd4ade6d, numTrees=20, numFeatures=7]


In [48]:
pred = cvModel.transform(test_data)


In [49]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol = 'MSRP')
rmse = eval.evaluate(pred, {eval.metricName:'rmse'})
mae = eval.evaluate(pred, {eval.metricName:"mae"})
r2 =eval.evaluate(pred,{eval.metricName:'r2'})

In [50]:
print("RMSE: %.3f" %rmse)
print("MAE: %.3f" %mae)
print("R2: %.3f" %r2)

RMSE: 37157.196
MAE: 9726.347
R2: 0.768
